In [ ]:
#!pip install -U google-generativeai
#!pip install -U google-generativeai gspread pandas gspread_dataframe


In [ ]:
import google.generativeai as genai
import pandas as pd

genai.configure(api_key="[Its a secret bro (use your own API Key)]")


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload file credentials Google Cloud kamu (format .json)


Saving black-outlet-416704-b8b8388debe4.json to black-outlet-416704-b8b8388debe4.json


In [ ]:
import gspread
from gspread_dataframe import get_as_dataframe
import pandas as pd
import google.generativeai as genai

# 🔑 Ganti dengan nama file kredensial kamu yang di-upload
SERVICE_ACCOUNT_FILE = "[my file.json]"

# 🔑 Ganti dengan ID spreadsheet kamu
SPREADSHEET_ID = "[It's also a secret bro]"
SHEET_NAME = "Sheet1"

# ✅ Koneksikan ke Google Sheets
gc = gspread.service_account(filename=SERVICE_ACCOUNT_FILE)
spreadsheet = gc.open_by_key(SPREADSHEET_ID)
worksheet = spreadsheet.worksheet(SHEET_NAME)

# ✅ Konfigurasi Gemini
genai.configure(api_key="[Its a secret bro (use your own API Key)]")  # Ganti dengan API key milikmu
model = genai.GenerativeModel("gemini-1.5-pro")


In [ ]:
import pandas as pd
import ast
import re
import time  # ✅ Tambahkan ini
import google.generativeai as genai

# 🔢 Tentukan baris awal dan akhir (baris pertama = 1)
start_row = 2
end_row = 101

# ✅ Ambil semua data sebagai DataFrame
all_data = get_as_dataframe(worksheet)
all_data.columns = all_data.columns.str.strip()
all_data = all_data.reset_index(drop=True)

# 🪄 Filter baris sesuai yang diinginkan
df_raw = all_data.iloc[start_row - 2:end_row - 1]
df_raw.columns = df_raw.columns.str.strip()
df_raw = df_raw.reset_index(drop=True)

# ✅ Bersihkan kolom Comment jika ada
if 'Comment' in df_raw.columns:
    df_raw['Comment'] = df_raw['Comment'].fillna("").astype(str).str.strip()
else:
    raise Exception("Kolom 'Comment' tidak ditemukan di sheet.")

# ✅ Daftar kategori klasifikasi
kategori_list = [
    "Curriculum or Learning Method", "Support for Home Learning", "Teacher Issue",
    "Learning Material & Study Tools", "Learning Distraction", "CCTV for Parents",
    "Crowded Waiting Room", "Facility Safety Concern", "Inadequate Classroom Facilities",
    "Inadequate Common Facilities", "Inadequate Prayer Room", "Parking Issue",
    "Toilet Cleanliness", "Additional Class/Schedule Request", "Feedback Positive",
    "Pricing or Promo Issue", "Class Timing & Punctuality", "Administrative Communication Issues",
    "Parent Participation Flexibility", "Progress Report Clarity", "Lesson Documentation Sharing",
    "Additional Operational Staff Support", "Class Transition & Flow/mobilisasi Management", "Uncategorized"
]
kategori_str = "\n- " + "\n- ".join(kategori_list)

# ✅ Fungsi membersihkan hasil dari Gemini

def bersihkan_jawaban(jawaban_mentah):
    cocok = re.search(r'\[\s*\[.*?\]\s*\]', jawaban_mentah, re.DOTALL)
    return cocok.group(0) if cocok else None

# ✅ Batch processing
batch_size = 10
rows_expanded = []

for batch_start in range(0, len(df_raw), batch_size):
    batch_df = df_raw.iloc[batch_start:batch_start + batch_size].copy()

    komentar_lines = []
    komentar_kosong_flags = []

    for i, komentar in enumerate(batch_df['Comment'], 1):
        komentar_lower = komentar.strip().lower()
        is_kosong = not komentar or komentar_lower in ['-', 'tidak ada', 'n/a']
        komentar_kosong_flags.append(is_kosong)
        komentar_lines.append(f"{i}. [KOSONG]" if is_kosong else f"{i}. {komentar.strip()}")

    # ✅ Prompt
    prompt = f"""
Saya memiliki daftar komentar dari orang tua murid. Tugas Anda adalah mengklasifikasikan **setiap komentar** ke dalam satu atau lebih dari kategori berikut:

{kategori_str}

**Jika komentar kosong, tidak diisi, hanya simbol (-), atau serupa**, maka hasilkan: **[]** (list kosong).

Format keluaran HARUS Python list of lists, TANPA penjelasan tambahan:

[
  ["Feedback Positive"],
  [],
  ["Crowded Waiting Room"],
  ...
]

Jumlah komentar: {len(batch_df)}. Pastikan jumlah list juga {len(batch_df)}.

Komentar:
{chr(10).join(komentar_lines)}
""".strip()

    # ✅ Kirim ke Gemini dan parsing

    try:
        response = model.generate_content(prompt)
        time.sleep(5)
        jawaban = response.text.strip()
        jawaban_bersih = bersihkan_jawaban(jawaban)
        if not jawaban_bersih:
            raise ValueError("❌ Tidak ditemukan list valid di hasil Gemini.")
        kategori_batch = ast.literal_eval(jawaban_bersih)
    except Exception as e:
        print(f"❌ Gagal parsing hasil Gemini untuk batch {batch_start}-{batch_start + batch_size}: {e}")
        kategori_batch = [[] if flag else ["Uncategorized"] for flag in komentar_kosong_flags]

    # ✅ Bangun hasil akhir
    for i, kategori_list_hasil in enumerate(kategori_batch):
        komentar_asli = batch_df.iloc[i]['Comment'].strip()
        row = batch_df.iloc[i].copy()

        if not kategori_list_hasil:
            row['Category'] = None
            rows_expanded.append(row)
        else:
            for kategori in kategori_list_hasil:
                if kategori not in kategori_list:
                    kategori = "Uncategorized"
                row_copy = row.copy()
                row_copy['Category'] = kategori
                rows_expanded.append(row_copy)

# ✅ Final dataframe hasil klasifikasi
df_kategori = pd.DataFrame(rows_expanded)


In [ ]:
df_kategori


In [ ]:
import pandas as pd

# Simpan df_final sebagai Excel
file_path = "df_final.xlsx"
df_kategori.to_excel(file_path, index=False)

# Jika di Colab: unduh file
from google.colab import files
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Update Live to Sheet

In [ ]:
def update_gsheet_with_df_final(worksheet, df_final, start_row=5, end_row=None, student_col_name="Student Name"):
    from gspread_dataframe import set_with_dataframe

    # Ambil semua data dari worksheet
    all_data = worksheet.get_all_values()
    header = all_data[0]

    # Batas akhir baris data yang diperiksa
    if end_row is None:
        end_row = len(all_data)

    # Ambil data yang ingin diperiksa dari baris start_row sampai end_row
    existing_data = all_data[start_row - 1:end_row]

    # Ambil indeks kolom student name
    try:
        student_col_index = header.index(student_col_name)
    except ValueError:
        raise Exception(f"Kolom '{student_col_name}' tidak ditemukan di header.")

    # Buat set nama student di df_final
    df_final_names = set(df_final[student_col_name].str.strip().str.lower())

    # Deteksi baris di GSheet yang perlu dihapus karena nama student sama
    rows_to_delete = []
    for i, row in enumerate(existing_data):
        if len(row) > student_col_index:
            name = row[student_col_index].strip().lower()
            if name in df_final_names:
                rows_to_delete.append(start_row + i)

    # Hapus baris-baris dengan nama yang sama (dari bawah ke atas)
    for row_num in sorted(rows_to_delete, reverse=True):
        worksheet.delete_rows(row_num)

    # Sisipkan baris kosong sebagai tempat df_final
    empty_row = [""] * len(df_final.columns)
    worksheet.insert_rows([empty_row] * len(df_final), row=start_row)

    # Tulis df_final ke worksheet
    from gspread_dataframe import set_with_dataframe
    set_with_dataframe(worksheet, df_final, row=start_row, include_column_header=False)

    print(f"✅ Berhasil update GSheet dari baris {start_row} sampai {end_row}.")
    print(f"❌ Dihapus: {len(rows_to_delete)} baris dengan student name sama.")
    print(f"✅ Disisipkan: {len(df_final)} baris dari df_final.")


In [ ]:
update_gsheet_with_df_final(worksheet, df_final, start_row=24373, end_row=24432)